# Create atomic files 
- User-item interactions
    - Raw data (.csv) -> Atomic files (.inter)
- Item features
    - Raw data (.csv) -> Atomic files (.itememb)

In [1]:
import pandas as pd
import numpy as np
import scipy.sparse as sp
import yaml
import os
import glob
import pickle
from tqdm import tqdm

In [2]:
# Raw data .csv files 
files = glob.glob('./dataset/transactions/*.csv')
files

['./dataset/transactions\\azuki.csv',
 './dataset/transactions\\bayc.csv',
 './dataset/transactions\\coolcats.csv',
 './dataset/transactions\\doodles.csv',
 './dataset/transactions\\meebits.csv']

In [3]:
# Get names which is the one before '.csv'
names = [os.path.basename(x).split('.')[0] for x in files]
names

['azuki', 'bayc', 'coolcats', 'doodles', 'meebits']

## .inter

In [5]:
for name, file in zip(tqdm(names), files):

    df_azuki = pd.read_csv(file)
    # df_azuki = df_azuki.drop_duplicates(subset=['Buyer', 'Token ID'], keep='first') # drop duplicated interactions
    user = df_azuki['Buyer'].values
    item = df_azuki['Token ID'].values

    save_path = './dataset/collections/' + name
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    f = open(save_path + f"/{name}.inter", 'w')
    f.write("user_id:token\titem_id:token\n")
    for i in range(len(user)):
        f.write("%s\t%d\n"%(user[i], item[i]))
    f.close()

100%|██████████| 5/5 [00:00<00:00, 15.41it/s]


## .itememb

### img, txt, price

In [11]:
for name in names:
    print('Collection name: ', name)
    for attribute in ['img', 'txt', 'price']:
        print('--- ', attribute)

        # Get raw data file
        file = pd.read_csv(f'./dataset/item_features/{name}_{attribute}.csv')
        print('before: ', file.shape)
        
        # 우리가 가진 기간 내 interaction에 등장하는 아이템만 남기기
        inter = pd.read_csv(f'./dataset/collections/{name}/{name}.inter', sep='\t')
        token_ids = inter['item_id:token'].unique()
        file = file[file['token_id'].isin(token_ids)].reset_index(drop=True)
        print('after: ', file.shape)

        save_path = './dataset/collections/' + name
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        
        f = open(save_path + f"/{name}.itememb_{attribute}", 'w')
        f.write(f"iid_{attribute}:token" + '\t' + f'item_emb_{attribute}:float_seq' + '\n')
        for i in tqdm(range(len(file))):
            # get token_id
            token_id = file['token_id'][i]
            # get the rest of the features
            features = file.iloc[i, 1:] # Series
            # write
            f.write(str(token_id) + '\t')
            for j in range(len(features)):
                f.write(f"{features[j].astype(np.float32)}") 
                # if it is not the last iteration
                if j != len(features) - 1:
                    f.write(' ')
            f.write('\n')    

        f.close()

Collection name:  azuki
---  img
before:  (10000, 65)
after:  (8386, 65)


100%|██████████| 8386/8386 [00:03<00:00, 2247.58it/s]


---  txt
before:  (10000, 1801)
after:  (8386, 1801)


100%|██████████| 8386/8386 [01:14<00:00, 112.69it/s]


---  price
before:  (8386, 2)
after:  (8386, 2)


...

100%|██████████| 4942/4942 [00:02<00:00, 2330.22it/s]


---  txt
before:  (20000, 1801)
after:  (5702, 1801)


100%|██████████| 5702/5702 [00:50<00:00, 112.40it/s]


---  price
before:  (5702, 2)
after:  (5702, 2)


100%|██████████| 5702/5702 [00:00<00:00, 7419.86it/s]
